# Imports and Configs

In [2]:
import mysql.connector
import os
import pandas as pd
import yfinance as yf
import yaml
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# Import config
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)
    start_date = config['start_date']

end_date = pd.to_datetime('today').date()

# Import benchmark config
with open("../portfolios/dfic_core.yaml", "r") as f:
    config = yaml.safe_load(f)
    securities = config['securities']
    currencies = config['currencies']
    transactions = config['transactions']

In [45]:
# Set up SQL database connection

connection = mysql.connector.connect(
    host=os.getenv('DB_HOSTNAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    port=os.getenv('DB_PORT'),
    database="Fund"
)

cursor = connection.cursor()

# Securities

In [46]:
# Create Securities SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Securities (
    ticker VARCHAR(10) PRIMARY KEY,
    name VARCHAR(100),
    type VARCHAR(20) NOT NULL,
    geography VARCHAR(50),
    sector VARCHAR(50),
    fund VARCHAR(50),
    currency CHAR(3) NOT NULL
);
""")
connection.commit()

In [47]:
# Backfill Securities Table

for security in securities:
    data = yf.Ticker(security['ticker'])
    fund = security['fund']
    sector = security['sector'] 
    name = data.info['longName']
    geography = security['geography'] 
    type = data.info['typeDisp']
    currency = data.info['currency']

    cursor.execute("""
    INSERT INTO Securities (ticker, name, type, geography, sector, fund, currency)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    ticker = VALUES(ticker),
    name = VALUES(name),
    type = VALUES(type),
    geography = VALUES(geography),
    sector = VALUES(sector),
    fund = VALUES(fund),
    currency = VALUES(currency);
    """, (security['ticker'], name, type, geography, sector, fund, currency))
    connection.commit()

In [42]:
# Drop Securities SQL Table

cursor.execute("DROP TABLE Securities")
connection.commit()

# Transactions

In [48]:
# Create Transactions SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    ticker VARCHAR(10) NOT NULL,
    date DATE NOT NULL,
    action ENUM('BUY', 'SELL') NOT NULL,
    shares INTEGER NOT NULL CHECK (shares > 0),
    price DECIMAL(20,10) NOT NULL CHECK (price > 0),
    currency CHAR(3) NOT NULL,
    FOREIGN KEY (ticker) REFERENCES Securities(ticker),
    UNIQUE (date, ticker, action)
);
""")
connection.commit()

In [49]:
# Backfill Transactions Table

for transaction in transactions:
    print(transaction['ticker'])
    ticker = transaction['ticker']
    date = transaction['date']
    action = transaction['type']
    shares = transaction['quantity']
    price = transaction['price']
    currency = transaction['currency']

    cursor.execute("""
    INSERT INTO Transactions (ticker, date, action, shares, price, currency)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
    shares = VALUES(shares), 
    price = VALUES(price), 
    currency = VALUES(currency);
    """, (ticker, date, action, shares, price, currency))
    connection.commit()

EA
ISRG
MA
TEX
AGG
RSP
AGG
AGG
RSP
HBM.TO
XIU.TO
XBB.TO
L.TO
SPY
APO
AAPL
CEG
AMSF
VEEV
SPY
GSL
AGG
SPSB
WFG
XIU.TO
CSH-UN.TO
XBB.TO
XIU.TO
ACO-X.TO
CG
WSC
AER
BLBD
TMUS
MP
DOLE
AMAT


In [40]:
# Drop Transactions SQL Table

cursor.execute("DROP TABLE Transactions")
connection.commit()

# Currencies

In [142]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Currencies (
    date DATE NOT NULL PRIMARY KEY,
    CAD DECIMAL(20,10)  DEFAULT 1.0,
    USD DECIMAL(20,10)  DEFAULT 1.0,
    EUR DECIMAL(20,10)  DEFAULT 1.0
);
""")
connection.commit()

In [143]:
# Backfill Currencies Table with corrected logic
for currency in currencies:
    ticker = currency['ticker']
    data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    data.index = data.index.date
    
    if currency['currency'] == 'USD':
        cursor.executemany("""
        INSERT INTO Currencies (date, CAD, USD, EUR)
        VALUES (%s, 1, %s, NULL)
        ON DUPLICATE KEY UPDATE
        USD = VALUES(USD);
        """, [(date, 1/rate) for date, rate in data.items()])
    elif currency['currency'] == 'EUR':
        cursor.executemany("""
        INSERT INTO Currencies (date, CAD, USD, EUR)
        VALUES (%s, 1, NULL, %s)
        ON DUPLICATE KEY UPDATE
        EUR = VALUES(EUR);
        """, [(date, 1/rate) for date, rate in data.items()])
    
    connection.commit()

In [141]:
# Drop Currencies SQL Table

cursor.execute("DROP TABLE Currencies")
connection.commit()

# Dates (old)

In [104]:
# Create Dates SQL Table

cursor.execute("""
CREATE TABLE IF NOT EXISTS Dates (
    date DATE PRIMARY KEY
);
""")
connection.commit()

In [ ]:
# Backfill Dates Table

sp500 = yf.Ticker('^GSPC').history(start=start_date, end=end_date)
tsx = yf.Ticker('^GSPTSE').history(start=start_date, end=end_date)

sp500.index = pd.to_datetime(sp500.index).tz_localize(None)
tsx.index = pd.to_datetime(tsx.index).tz_localize(None)

valid_dates = sp500.index.union(tsx.index)
valid_dates = [date.date() for date in valid_dates]

cursor.executemany("""
    INSERT INTO Dates (date)
    VALUES (%s)
    ON DUPLICATE KEY UPDATE date = VALUES(date);
""", [(date,) for date in valid_dates])
connection.commit()

In [110]:
# Drop Dates SQL Table

cursor.execute("DROP TABLE Dates")
connection.commit()

# Trading Calendar

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS TradingCalendar (
    trading_date DATE PRIMARY KEY,
    is_us_trading_day BOOLEAN NOT NULL,
    is_ca_trading_day BOOLEAN NOT NULL
);
""")
connection.commit()

In [70]:
# Backfill Dates Table

sp500 = yf.Ticker('^GSPC').history(start=start_date, end=end_date)
tsx = yf.Ticker('^GSPTSE').history(start=start_date, end=end_date)

sp500.index = pd.to_datetime(sp500.index).tz_localize(None)
tsx.index = pd.to_datetime(tsx.index).tz_localize(None)

# valid_dates = sp500.index.union(tsx.index)
# valid_dates = [date.date() for date in valid_dates]

# cursor.executemany("""
#     INSERT INTO Dates (date)
#     VALUES (%s)
#     ON DUPLICATE KEY UPDATE date = VALUES(date);
# """, [(date,) for date in valid_dates])
# connection.commit()

In [72]:
valid_US = [date.date() for date in sp500.index]
valid_CA = [date.date() for date in tsx.index]

valid_dates = sp500.index.union(tsx.index)
valid_dates = [date.date() for date in valid_dates]

for date in valid_dates:
    is_us_trading_day = date in valid_US
    is_ca_trading_day = date in valid_CA

    cursor.executemany("""
    INSERT INTO TradingCalendar (trading_date, is_us_trading_day, is_ca_trading_day)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE trading_date = VALUES(trading_date);
    """, [(date, is_us_trading_day, is_ca_trading_day)])
    connection.commit()


In [68]:
# Drop Dates SQL Table

cursor.execute("DROP TABLE TradingCalendar")
connection.commit()

# prices

In [105]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Prices (
    ticker VARCHAR(10) NOT NULL,
    trading_date DATE NOT NULL,
    price DECIMAL(20,10) NULL CHECK (price IS NULL OR price > 0),
    currency CHAR(3) NOT NULL,
    PRIMARY KEY (ticker, trading_date),
    FOREIGN KEY (ticker) REFERENCES Securities(ticker)
);
""")
connection.commit()

### Will leave price gaps for holidays

In [ ]:
cursor.execute("""
    SELECT ticker, date, action, shares
    FROM Transactions
    ORDER BY ticker, date
""")
transactions = cursor.fetchall()

# holding periods
holding_periods = {}
current_position = {}
current_ticker = None

for ticker, date, action, shares in transactions:
    if ticker != current_ticker:
        if current_ticker and current_position.get(current_ticker, 0) > 0:
            # add final
            end_date = pd.to_datetime('today').date()
            holding_periods[current_ticker][-1][1] = end_date
        
        current_ticker = ticker
        current_position[ticker] = 0
        holding_periods[ticker] = []
    
    position_change = shares if action == 'BUY' else -shares
    old_position = current_position[ticker]
    current_position[ticker] += position_change
    
    if old_position == 0 and current_position[ticker] > 0:
        # new holding period
        holding_periods[ticker].append([date, None])
    elif old_position > 0 and current_position[ticker] == 0:
        # end holding period
        holding_periods[ticker][-1][1] = date

# handle open positions
for ticker, position in current_position.items():
    if position > 0 and holding_periods[ticker][-1][1] is None:
        holding_periods[ticker][-1][1] = pd.to_datetime('today').date()

# backfill 
print("upd: ")
for ticker, periods in holding_periods.items():
    print(f"{ticker} ", end='', flush=True)
    
    try:
        cursor.execute("""
            SELECT currency 
            FROM Securities 
            WHERE ticker = %s
        """, (ticker,))
        currency = cursor.fetchone()[0]
        
        # each holding period
        for start_date, end_date in periods:
            # all trading days for this period
            cursor.execute("""
                SELECT trading_date 
                FROM TradingCalendar
                WHERE trading_date BETWEEN %s AND %s
                ORDER BY trading_date
            """, (start_date, end_date))
            
            trading_days = [row[0] for row in cursor.fetchall()]
            
            if not trading_days:
                continue
                
# check this part ------------------------------------------------------------------------------------------

            data = yf.Ticker(ticker).history(start=start_date, end=end_date)
            data.index = data.index.date
            
            price_data = []
            for trade_date in trading_days:
                if trade_date in data.index:
                    price = float(data.loc[trade_date, 'Close'])
                else:
                    price = None
                
                price_data.append((
                    ticker,
                    trade_date,
                    price,
                    currency
                ))
            
            # insert with NULL 
            cursor.executemany("""
                INSERT INTO Prices (ticker, trading_date, price, currency)
                VALUES (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    price = VALUES(price),
                    currency = VALUES(currency)
            """, price_data)
            
            connection.commit()
        
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        connection.rollback()

upd: 
AAPL ACO-X.TO AER AGG AMAT AMSF APO BLBD CEG CG CSH-UN.TO DOLE EA GSL HBM.TO ISRG L.TO MA MP RSP SPSB SPY TEX TMUS VEEV WFG WSC XBB.TO XIU.TO 

### frontfills holiday prices missing from previous backfill


In [108]:

cursor.execute("""
    SELECT DISTINCT ticker 
    FROM Prices 
    WHERE price IS NULL
""")
tickers_with_nulls = [row[0] for row in cursor.fetchall()]

print("Frontfilling prices for:", end=' ')

for ticker in tickers_with_nulls:
    print(f"{ticker}", end=' ', flush=True)
    

    cursor.execute("""
        WITH LastKnownPrice AS (
            SELECT 
                p1.ticker,
                p1.trading_date,
                (
                    SELECT p2.price 
                    FROM Prices p2 
                    WHERE p2.ticker = p1.ticker 
                        AND p2.trading_date < p1.trading_date 
                        AND p2.price IS NOT NULL 
                    ORDER BY p2.trading_date DESC 
                    LIMIT 1
                ) as last_price
            FROM Prices p1
            WHERE p1.ticker = %s AND p1.price IS NULL
        )
        UPDATE Prices p
        JOIN LastKnownPrice lkp ON p.ticker = lkp.ticker AND p.trading_date = lkp.trading_date
        SET p.price = lkp.last_price
        WHERE p.price IS NULL;
    """, (ticker,))
    
    connection.commit()

Frontfilling prices for: AAPL ACO-X.TO AER AGG AMAT AMSF APO BLBD CEG CG CSH-UN.TO DOLE EA GSL HBM.TO ISRG L.TO MA MP RSP SPSB SPY TEX TMUS VEEV WFG WSC XBB.TO XIU.TO 

In [107]:
holding_periods

{'AAPL': [[datetime.date(2023, 4, 27), datetime.date(2025, 3, 18)]],
 'ACO-X.TO': [[datetime.date(2024, 12, 23), datetime.date(2025, 3, 18)]],
 'AER': [[datetime.date(2024, 12, 23), datetime.date(2025, 3, 18)]],
 'AGG': [[datetime.date(2022, 5, 9), datetime.date(2023, 4, 27)],
  [datetime.date(2023, 4, 27), datetime.date(2025, 3, 18)]],
 'AMAT': [[datetime.date(2024, 12, 23), datetime.date(2025, 3, 18)]],
 'AMSF': [[datetime.date(2023, 12, 27), datetime.date(2025, 3, 18)]],
 'APO': [[datetime.date(2023, 4, 27), datetime.date(2025, 3, 18)]],
 'BLBD': [[datetime.date(2024, 12, 23), datetime.date(2025, 3, 18)]],
 'CEG': [[datetime.date(2023, 4, 27), datetime.date(2025, 3, 18)]],
 'CG': [[datetime.date(2024, 12, 23), datetime.date(2025, 3, 18)]],
 'CSH-UN.TO': [[datetime.date(2023, 12, 28), datetime.date(2025, 3, 18)]],
 'DOLE': [[datetime.date(2024, 12, 23), datetime.date(2025, 3, 18)]],
 'EA': [[datetime.date(2022, 5, 5), datetime.date(2025, 3, 18)]],
 'GSL': [[datetime.date(2023, 12, 27

In [104]:
# Drop Dates SQL Table

cursor.execute("DROP TABLE Prices")
connection.commit()

# Generate Holdings Table

In [109]:

cursor.execute("""
CREATE VIEW Holdings AS
SELECT
    p.trading_date,
    p.ticker,
    s.name,
    s.type,
    s.geography,
    s.sector,
    s.fund,
    s.currency AS security_currency,
    (
      SELECT SUM(
               CASE 
                 WHEN t.action = 'BUY' THEN t.shares 
                 ELSE -t.shares 
               END
             )
      FROM Transactions t
      WHERE t.ticker = p.ticker 
        AND t.date <= p.trading_date
    ) AS shares_held,
    p.price,
    (
      (
        SELECT SUM(
                 CASE 
                   WHEN t.action = 'BUY' THEN t.shares 
                   ELSE -t.shares 
                 END
               )
        FROM Transactions t
        WHERE t.ticker = p.ticker 
          AND t.date <= p.trading_date
      ) * p.price
    ) AS market_value
FROM Prices p
JOIN Securities s ON s.ticker = p.ticker;
""")
connection.commit()